# Conclusion
I was wrong. Generation of word pairs ignore the condition that the inner masks are symmetric. 

## todo
- relax punctuation

From https://czechtheworld.com/best-palindromes/#palindromes-symmetric-by-words

- You can cage a swallow, can’t you, but you can’t swallow a cage, can you?
- Fall leaves as soon as leaves fall.
- King, are you glad you are king?
- So patient a nurse to nurse a patient so.
- First ladies rule the state, and state the rule: “Ladies First!”
- Do I, like, look like I do?
- Sorry, I am very awkward. Very am I sorry.
- Is it crazy how saying sentences backwards creates backwards sentences saying how crazy it is?

In [1]:
from random import random
import numpy as np
import scipy
from transformers import AutoTokenizer, BertForMaskedLM
import torch

In [ ]:
uncasedTok = AutoTokenizer.from_pretrained("bert-base-uncased")
casedTok = AutoTokenizer.from_pretrained("bert-base-cased")
uncasedBert = BertForMaskedLM.from_pretrained("bert-base-uncased")
casedBert = BertForMaskedLM.from_pretrained("bert-base-cased")
CASED = 'CASED'
UNCASED = 'UNCASED'

In [3]:
def getMode(logits):
    return logits[0].argmax(dim=1)

In [4]:
def extend(casing, response_len, prompt, ends_with):
    if casing is CASED:
        bert = casedBert
        tok = casedTok
    else:
        bert = uncasedBert
        tok = uncasedTok
    input_idx = tok.encode(prompt.strip(' '))
    assert input_idx[-1] == tok.sep_token_id
    input_idx.pop(-1)
    len_input = len(input_idx)
    input_idx += (
        [tok.mask_token_id] * response_len + 
        [tok.convert_tokens_to_ids(ends_with)] + 
        [tok.sep_token_id]
    )
    print(tok.decode(input_idx))
    buffer = input_idx[:]
    
    for i in range(response_len):
        logits = bert(torch.tensor([buffer]))[0]
        prediction = getMode(logits)
        buffer[len_input + i] = prediction[len_input + i]
#         print(i, tok.decode(buffer))
    print()
    print(tok.decode(buffer))

In [21]:
extend(UNCASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '?')
print()
extend(UNCASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '.')
print()
extend(CASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '?')
print()
extend(CASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '.')

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? how long is it going to take? and how long will it take to arrive? [SEP]

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP]

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? " asked the american. " yes, " he replied, " it will not. [SEP]

[CLS] Nobody expected the cancellation of the flight, so people began to panic. What if the r

In [23]:
extend(UNCASED, 16, '', '?')

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]

[CLS] " what the hell are you doing, " he said, " getting a gun? [SEP]


In [27]:
extend(CASED, 16, 'In reality, my dog', '.')

[CLS] In reality, my dog [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP]

[CLS] In reality, my dog is the only - the - best - thing that could ever be in my life. [SEP]


In [ ]:
# def proposeReversal(prompt):
#     words = [*reversed(prompt.split(' '))]
#     return ' '.join(words)

In [163]:
def crab(response_len, prompt, ends_with, sampleMethod, casing, seed=None):
    if casing is CASED:
        bert = casedBert
        tok = casedTok
    else:
        bert = uncasedBert
        tok = uncasedTok
    input_idx = tok.encode(prompt.strip(' '))[1:-1]
    buffer = [None] * (
        1 + len(input_idx) + response_len + len(input_idx) + 1 + 1
    )
    acc = 0
    buffer[acc] = tok.cls_token_id
    acc += 1
    buffer[acc : acc + len(input_idx)] = input_idx
    acc += len(input_idx)
    buffer[acc : acc + response_len] = [tok.mask_token_id] * response_len
    acc += response_len
    buffer[acc : acc + len(input_idx)] = reversed(input_idx)
    acc += len(input_idx)
    if ends_with:
        buffer[acc] = tok.convert_tokens_to_ids(ends_with)
        acc += 1
    else:
        buffer.pop(-1)
    buffer[acc] = tok.sep_token_id
    acc += 1
    assert acc == len(buffer)
    
    print(tok.decode(buffer))
    for i in range((response_len + 1) // 2):
#         print(i, tok.decode(buffer))
        left  = 1 + len(input_idx) + i
        right = 1 + len(input_idx) + response_len - i - 1
        if seed is not None:
            pick = seed[i]
        else:
            pick = 0
        word_id = sampleMethod(buffer, left, right, pick, bert, tok)
        buffer[left] = word_id
        buffer[right] = word_id
#     print()
    print(tok.decode(buffer))

In [164]:
def fastest(buffer, left, right, _, bert, tok):
    logits = bert(torch.tensor([buffer]))[0].detach()
    logit = (
        scipy.special.softmax(logits[0][left]) * 
        scipy.special.softmax(logits[0][right])
    )
    return logit.argmax()

Optimization: stop looking when left prob < current best shot

In [186]:
def slowest(buffer, left, right, pick, bert, tok):
    logits = bert(torch.tensor([buffer]))[0].detach()
    left_probs = scipy.special.softmax(logits[0][left])
    left_rank = np.argsort(left_probs)
    probs = []
    max_prob = [0]
    last_print = None
    for i, word_id in enumerate(reversed(left_rank)):
#         print(i)
        left_prob = left_probs[word_id]
#         print(left_prob)
        if left_prob < max_prob[-1]:
            print('\n  Early stopping at', i, 'which is', i / len(left_probs))
            break
        buffer[left] = word_id
        logits = bert(torch.tensor([buffer]))[0].detach()
        right_probs = scipy.special.softmax(logits[0][right])
        prob = left_prob * right_probs[word_id]
#         print('*', right_probs[word_id], '=', prob)
        probs.append((word_id, prob))
        probs.sort(key=lambda x:x[1])
        max_prob.append(prob)
        max_prob.sort(reverse = True)
        max_prob = max_prob[:pick + 1]
        if last_print != max_prob[0]:
            last_print = max_prob[0]
            print(
                '  Current best: "', tok.convert_ids_to_tokens(
                    probs[-1]
                )[0], '"...', end = '\r', flush = True, 
            )
    s = sorted(probs, key=lambda x:x[1])
    return s[-1 - pick][0]

In [93]:
# TAIL_CUT = 0
# def slowestRand(buffer, left, right):
#     ...

In [170]:
crab(1, 'Fall leaves as', '.', slowest, UNCASED)
print()
crab(1, 'Fall leaves as', '.', slowest, CASED, [3])

[CLS] fall leaves as [MASK] as leaves fall. [SEP]
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] fall leaves as soon as leaves fall. [SEP]

[CLS] Fall leaves as [MASK] as leaves Fall. [SEP]
  Early stopping at 4 which is 0.00013795006207752792
[CLS] Fall leaves as easily as leaves Fall. [SEP]


In [95]:
crab(3, 'Fall leaves', '.', slowest, UNCASED, [1, 0])

[CLS] fall leaves [MASK] [MASK] [MASK] leaves fall. [SEP]
  Early stopping at 30 which is 0.000982897582071948
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] fall leaves falling. falling leaves fall. [SEP]


In [96]:
crab(5, 'Fall', '.', slowest, UNCASED, [5, 0, 0])

[CLS] fall [MASK] [MASK] [MASK] [MASK] [MASK] fall. [SEP]
  Early stopping at 20 which is 0.0006552650547146321
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] fall = fall. fall = fall. [SEP]


In [97]:
crab(1, 'So patient a nurse', '.', slowest, UNCASED, [5])

[CLS] so patient a nurse [MASK] nurse a patient so. [SEP]
  Early stopping at 6 which is 0.00019657951641438963
[CLS] so patient a nurse to nurse a patient so. [SEP]


In [98]:
crab(3, 'So patient a', '.', slowest, UNCASED, [2, 0])

[CLS] so patient a [MASK] [MASK] [MASK] a patient so. [SEP]
  Early stopping at 25 which is 0.0008190813183932901
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] so patient a doctor a doctor a patient so. [SEP]


In [99]:
crab(
    1, 'Is it crazy how saying sentences backwards', '?', slowest, 
    UNCASED, [9], 
)

[CLS] is it crazy how saying sentences backwards [MASK] backwards sentences saying how crazy it is? [SEP]
  Early stopping at 10 which is 0.00032763252735731603
[CLS] is it crazy how saying sentences backwards makes backwards sentences saying how crazy it is? [SEP]


In [100]:
# crab(
#     3, 'Is it crazy how saying sentences', '?', slowest, 
#     UNCASED, [0, 9-9], 
# )

Junyan recommendation: outfilling
## Outfilling

In [172]:
def outfill(response_len, prompt, ends_with, sampleMethod, casing, seed=None):
    if casing is CASED:
        bert = casedBert
        tok = casedTok
    else:
        bert = uncasedBert
        tok = uncasedTok
    input_idx = tok.encode(prompt.strip(' '))[1:-1]
    buffer = [None] * (
        1 + response_len + len(input_idx) + response_len + 1 + 1
    )
    acc = 0
    buffer[acc] = tok.cls_token_id
    acc += 1
    buffer[acc : acc + response_len] = [tok.mask_token_id] * response_len
    acc += response_len
    buffer[acc : acc + len(input_idx)] = input_idx
    acc += len(input_idx)
    buffer[acc : acc + response_len] = [tok.mask_token_id] * response_len
    acc += response_len
    if ends_with:
        buffer[acc] = tok.convert_tokens_to_ids(ends_with)
        acc += 1
    else:
        buffer.pop(-1)
    buffer[acc] = tok.sep_token_id
    acc += 1
    assert acc == len(buffer)
    
    print(tok.decode(buffer))
    for i in range(response_len):
#         print(i, tok.decode(buffer))
        left  = 1 + response_len - i - 1
        right = 1 + response_len + len(input_idx) + i
        if seed is not None:
            pick = seed[i]
        else:
            pick = 0
        word_id = sampleMethod(buffer, left, right, pick, bert, tok)
        buffer[left] = word_id
        buffer[right] = word_id
#     print()
    print(tok.decode(buffer))

In [12]:
outfill(1, 'leaves as soon as leaves', '.', slowest, UNCASED)

[CLS] [MASK] leaves as soon as leaves [MASK]. [SEP]
  Early stopping at 70 which is 0.0022934276915012124
[CLS] he leaves as soon as leaves he. [SEP]


In [14]:
outfill(2, 'as soon as', '.', fastest, UNCASED)
# slowest never returned...

[CLS] [MASK] [MASK] as soon as [MASK] [MASK]. [SEP]
[CLS] i said as soon as said i. [SEP]


In [20]:
outfill(3, 'soon', '.', slowest, UNCASED, [0, 2, 1])

[CLS] [MASK] [MASK] [MASK] soon [MASK] [MASK] [MASK]. [SEP]
  Early stopping at 24 which is 0.0007863180656575585
  Early stopping at 26 which is 0.0008518445711290217
  Early stopping at 41 which is 0.0013432933621649957
[CLS] someday maybe, soon, maybe someday. [SEP]


In [27]:
outfill(
    1, 'patient a nurse to nurse a patient', '.', 
    slowest, UNCASED, [5], 
)

[CLS] [MASK] patient a nurse to nurse a patient [MASK]. [SEP]
  Early stopping at 162 which is 0.0053076469431885195
[CLS] pet patient a nurse to nurse a patient pet. [SEP]


In [33]:
outfill(
    2, 'a nurse to nurse a', '.', 
    fastest, UNCASED, [0, 0], 
)

[CLS] [MASK] [MASK] a nurse to nurse a [MASK] [MASK]. [SEP]
[CLS] someone needs a nurse to nurse a needs someone. [SEP]


In [39]:
outfill(
    4, 'to', '.', 
    slowest, UNCASED, [1, 0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] to [MASK] [MASK] [MASK] [MASK]. [SEP]
  Early stopping at 484 which is 0.015857414324094096
  Early stopping at 73 which is 0.002391717449708407
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 7 which is 0.00022934276915012123
[CLS] baby, you have to have you, baby. [SEP]


In [40]:
outfill(
    1, 'it crazy how saying sentences backwards creates backwards sentences saying how crazy it', 
    '?', 
    slowest, UNCASED, [0], 
)

[CLS] [MASK] it crazy how saying sentences backwards creates backwards sentences saying how crazy it [MASK]? [SEP]
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] is it crazy how saying sentences backwards creates backwards sentences saying how crazy it is? [SEP]


In [43]:
outfill(
    2, 'crazy how saying sentences backwards creates backwards sentences saying how crazy', 
    '?', 
    slowest, UNCASED, [1, 0], 
)

[CLS] [MASK] [MASK] crazy how saying sentences backwards creates backwards sentences saying how crazy [MASK] [MASK]? [SEP]
  Early stopping at 141 which is 0.004619618635738156
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] are you crazy how saying sentences backwards creates backwards sentences saying how crazy you are? [SEP]


In [44]:
outfill(
    2, 'crazy how saying sentences backwards creates backwards sentences saying how crazy', 
    '?', 
    slowest, UNCASED, [2, 0], 
)

[CLS] [MASK] [MASK] crazy how saying sentences backwards creates backwards sentences saying how crazy [MASK] [MASK]? [SEP]
  Early stopping at 2 which is 6.552650547146321e-05
  Early stopping at 2 which is 6.552650547146321e-05
[CLS] it is crazy how saying sentences backwards creates backwards sentences saying how crazy is it? [SEP]


In [46]:
outfill(
    2, 'crazy how saying sentences backwards creates backwards sentences saying how crazy', 
    '?', 
    slowest, UNCASED, [4, 0], 
)

[CLS] [MASK] [MASK] crazy how saying sentences backwards creates backwards sentences saying how crazy [MASK] [MASK]? [SEP]
  Early stopping at 163 which is 0.005340410195924251
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] words are crazy how saying sentences backwards creates backwards sentences saying how crazy are words? [SEP]


In [48]:
outfill(
    3, 'how saying sentences backwards creates backwards sentences saying how', 
    '?', 
    fastest, UNCASED, [0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] how saying sentences backwards creates backwards sentences saying how [MASK] [MASK] [MASK]? [SEP]
[CLS] so how about how saying sentences backwards creates backwards sentences saying how about how so? [SEP]


In [52]:
outfill(
    4, 'saying sentences backwards creates backwards sentences saying', 
    '?', 
    fastest, UNCASED, [0, 0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] saying sentences backwards creates backwards sentences saying [MASK] [MASK] [MASK] [MASK]? [SEP]
[CLS] backwards sentences saying sentences saying sentences backwards creates backwards sentences saying sentences saying sentences backwards? [SEP]


In [60]:
outfill(
    4, 'saying sentences backwards creates backwards sentences saying', 
    '?', 
    slowest, UNCASED, [2, 0, 3, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] saying sentences backwards creates backwards sentences saying [MASK] [MASK] [MASK] [MASK]? [SEP]
  Early stopping at 29 which is 0.0009501343293362165
  Early stopping at 14 which is 0.00045868553830024245
  Early stopping at 15 which is 0.000491448791035974
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] " hello ", saying sentences backwards creates backwards sentences saying, " hello "? [SEP]


In [61]:
outfill(
    5, 'sentences backwards creates backwards sentences', 
    '?', 
    slowest, UNCASED, [0, 0, 0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] sentences backwards creates backwards sentences [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]
  Early stopping at 26 which is 0.0008518445711290217
  Early stopping at 4 which is 0.00013105301094292642
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] backwards creates backwards sentences. sentences backwards creates backwards sentences. sentences backwards creates backwards? [SEP]


In [64]:
outfill(
    5, 'sentences backwards creates backwards sentences', 
    '?', 
    slowest, UNCASED, [0, 2, 0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] sentences backwards creates backwards sentences [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]
  Early stopping at 26 which is 0.0008518445711290217
  Early stopping at 35 which is 0.0011467138457506062
  Early stopping at 21 which is 0.0006880283074503637
  Early stopping at 2 which is 6.552650547146321e-05
  Early stopping at 6 which is 0.00019657951641438963
[CLS] backwards words are words. sentences backwards creates backwards sentences. words are words backwards? [SEP]


In [93]:
outfill(
    7, 'creates', 
    '?', 
    slowest, UNCASED, [2, 0, 0, 0, 0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] creates [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]
  Early stopping at 54 which is 0.0017692156477295065
  Early stopping at 703 which is 0.023032566673219318
  Early stopping at 8 which is 0.00026210602188585284
  Early stopping at 14 which is 0.00045868553830024245
  Early stopping at 15 which is 0.000491448791035974
  Early stopping at 29 which is 0.0009501343293362165
  Early stopping at 3 which is 9.828975820719482e-05
[CLS] what is what is that thing that creates that thing that is what is what? [SEP]


In [98]:
outfill(
    5, 'creates', 
    '?', 
    slowest, UNCASED, [3, 0, 1, 0, 0, 0, 0], 
)

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] creates [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]
  Early stopping at 110 which is 0.0036039578009304765
  Early stopping at 63 which is 0.0020640849223510912
  Early stopping at 8 which is 0.00026210602188585284
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 6 which is 0.00019657951641438963
[CLS] what is that thing that creates that thing that is what? [SEP]


In [101]:
outfill(
    4, 'means', 
    '.', 
    slowest, UNCASED, 
)

[CLS] [MASK] [MASK] [MASK] [MASK] means [MASK] [MASK] [MASK] [MASK]. [SEP]
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 5 which is 0.00016381626367865801
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 38 which is 0.0012450036039578008
[CLS] letter " a " means " a " letter. [SEP]


In [102]:
outfill(
    5, 'means', 
    '.', 
    slowest, UNCASED, 
)

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] means [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP]
  Early stopping at 2 which is 6.552650547146321e-05
  Early stopping at 8 which is 0.00026210602188585284
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 9 which is 0.00029486927462158444
  Early stopping at 312 which is 0.01022213485354826
[CLS] ending in " a " means " a " in ending. [SEP]


In [149]:
outfill(
    3, 'only', 
    '.', 
    slowest, UNCASED, 
)

[CLS] [MASK] [MASK] [MASK] only [MASK] [MASK] [MASK]. [SEP]
  Early stopping at 113 which is 0.003702247559137671
  Early stopping at 40 which is 0.0013105301094292641
  Early stopping at 326 which is 0.010680820391848502
[CLS] division one had only had one division. [SEP]


In [197]:
def slowestNoPunc(buffer, left, right, pick, bert, tok):
    logits = bert(torch.tensor([buffer]))[0].detach()
    left_probs = scipy.special.softmax(logits[0][left])
    left_rank = np.argsort(left_probs)
    probs = []
    max_prob = [0]
    last_print = None
    for i, word_id in enumerate(reversed(left_rank)):
#         print(i)
        if not tok.convert_ids_to_tokens([word_id])[0].isalpha():
            continue
        left_prob = left_probs[word_id]
#         print(left_prob)
        if left_prob < max_prob[-1]:
            print('\n  Early stopping at', i, 'which is', i / len(left_probs))
            break
        buffer[left] = word_id
        logits = bert(torch.tensor([buffer]))[0].detach()
        right_probs = scipy.special.softmax(logits[0][right])
        prob = left_prob * right_probs[word_id]
#         print('*', right_probs[word_id], '=', prob)
        probs.append((word_id, prob))
        probs.sort(key=lambda x:x[1])
        max_prob.append(prob)
        max_prob.sort(reverse = True)
        max_prob = max_prob[:pick + 1]
        if last_print != max_prob[0]:
            last_print = max_prob[0]
            print(
                '  Current best: "', tok.convert_ids_to_tokens(
                    probs[-1]
                )[0], '"...', end = '\r', flush = True, 
            )
    s = sorted(probs, key=lambda x:x[1])
    return s[-1 - pick][0]

In [210]:
outfill(
    5, 'but', 
    '.', 
    slowestNoPunc, UNCASED, 
)

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] but [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP]
  Current best: " it "....
  Early stopping at 77 which is 0.0025227704606513336
  Current best: " felt "...
  Early stopping at 166 which is 0.0054386999541314466
  Current best: " i "...
  Early stopping at 58 which is 0.001900268658672433
  Current best: " was ".......
  Early stopping at 14 which is 0.00045868553830024245
  Current best: " it ".....
  Early stopping at 12 which is 0.00039315903282877927
[CLS] it was i felt it but it felt i was it. [SEP]
